<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/tiny_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiny-ImageNet
It contains 200 classes from full imagenet dataset.<br>
However, file structures are not for train and validation images. Train images are separated with a class-wise folder name where all validation images are a single folder. We need an additional propcessing to rearrange validation images into class-wise folders.

Donwload & unzip

In [1]:
! wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
! unzip -q tiny-imagenet-200.zip

--2022-04-29 17:28:43--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  50.9MB/s    in 5.7s    

2022-04-29 17:28:49 (41.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



Training CNN model

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--train_batch', default=512, type=int, help='batch size')
parser.add_argument('--valid_batch', default=1024, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=200, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

Training

In [ ]:
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset
import glob
from PIL import Image

seed_everything()
VALID_DIR = 'tiny-imagenet-200/val'
TRAIN_DIR = 'tiny-imagenet-200/train'

class TinyImageNetDataset(Dataset):
    def __init__(self, root, id, transform=None, train=False):
        self.transform = transform
        self.id_dict = id
        self.train = train
        if self.train:
            self.filenames = glob.glob(os.path.join(root, "train/*/*/*.JPEG")) 
        else:
            self.filenames = glob.glob(os.path.join(root,"val/images/*.JPEG"))
            self.cls_dic = {}
            for i, line in enumerate(open(os.path.join(root,'val/val_annotations.txt'), 'r')):
                a = line.split('\t')
                img, cls_id = a[0],a[1]
                self.cls_dic[img] = id[cls_id]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = Image.open(img_path).convert('RGB')

        if self.train:
            label = self.id_dict[img_path.split('/')[4]]
        else:
            label = self.cls_dic[img_path.split('/')[-1]]

        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([transforms.Resize(64), transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )
transform_test = transforms.Compose([transforms.Resize(64), transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )

id_dict = {}
for i, line in enumerate(open('/content/tiny-imagenet-200/wnids.txt', 'r')):
  id_dict[line.replace('\n', '')] = i

root = '/content/tiny-imagenet-200/'
train_dataset = TinyImageNetDataset(root=root, id=id_dict, transform=transform_train, train=True)
test_dataset = TinyImageNetDataset(root=root, id=id_dict, transform=transform_test, train=False)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.train_batch, shuffle=True,
                                          num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.valid_batch, shuffle=False, 
                                         num_workers=2, pin_memory=True)
print('sample size- Train:%d, Validation:%d'%(len(train_dataset), len(test_dataset)))


model = models.resnet50(pretrained=True).to(device)
model.fc = torch.nn.Linear(model.fc.in_features, args.num_classes)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
best_epoch, best_acc = 0, 0
for epoch in range(args.num_epoch):
    for epoch in range(args.num_epoch):
        if epoch != 0 and epoch < 100 and epoch % 30 == 0:
            for param in optimizer.param_groups:
                param['lr'] = param['lr'] / 10 
        train(model, trainloader, criterion, optimizer)
        accuracy = test(model, testloader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_tiny_imagenet.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

sample size- Train:100000, Validation:10000
